Here's a detailed explanation of each part of the code:

### 1. **Importing Required Libraries:**
```python
import cv2
import mediapipe as mp
```
- **OpenCV (cv2)**: Used for capturing video frames from the webcam and displaying the output.
- **MediaPipe (mp)**: Google's framework for multimodal perception, used here to detect hand landmarks.

### 2. **Initializing MediaPipe Hands and Drawing Utilities:**
```python
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
```
- **mp_hands**: We initialize MediaPipe's hand detection solution.
- **mp_drawing**: MediaPipe provides utilities to draw landmarks on images.

### 3. **Gesture Classification Function:**
```python
def classify_hand_gesture(hand_landmarks):
    # We will use the tip of the fingers (landmarks: 8, 12, 16, 20) and the thumb (landmark: 4)
    # For simplicity, let's define:
    # Rock: All fingers are closed (folded)
    # Paper: All fingers are open
    # Scissors: Index and middle finger are open, others are closed
```
- This function classifies a hand gesture based on the detected hand landmarks (specific points on the hand).
- Each finger's tip has a specific landmark number. For example:
  - **Thumb**: 4
  - **Index**: 8
  - **Middle**: 12
  - **Ring**: 16
  - **Pinky**: 20
- **Rock**: All fingers are closed (i.e., the tips are below the base joints).
- **Paper**: All fingers are open (i.e., the tips are above the base joints).
- **Scissors**: Only the index and middle fingers are open, while the rest are closed.

```python
    thumb_tip = hand_landmarks.landmark[4].y
    index_tip = hand_landmarks.landmark[8].y
    middle_tip = hand_landmarks.landmark[12].y
    ring_tip = hand_landmarks.landmark[16].y
    pinky_tip = hand_landmarks.landmark[20].y
```
- This part retrieves the **Y-coordinate** of the fingertip positions (landmarks) to determine whether a finger is open or closed.
  - A lower Y value (closer to the top of the screen) indicates that the finger is open.
  
```python
    index_base = hand_landmarks.landmark[5].y
    middle_base = hand_landmarks.landmark[9].y
    ring_base = hand_landmarks.landmark[13].y
    pinky_base = hand_landmarks.landmark[17].y
```
- This retrieves the Y-coordinates of the base joints (second joints from the tips) of each finger to compare with the fingertip positions.

```python
    # Determine if fingers are open (tip is above the base) or closed
    thumb_open = hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x  # Thumb is open if x position of tip is less than the joint
    index_open = index_tip < index_base
    middle_open = middle_tip < middle_base
    ring_open = ring_tip < ring_base
    pinky_open = pinky_tip < pinky_base
```
- **thumb_open**: The thumb's openness is determined by checking the **X-coordinate** because the thumb moves sideways (along the X-axis), unlike other fingers.
- For other fingers, openness is determined by comparing the Y-coordinates of the tip and base joints. If the tip is higher (smaller Y value), the finger is considered open.

```python
    # Classify gesture
    if not thumb_open and not index_open and not middle_open and not ring_open and not pinky_open:
        return "Rock"
    elif index_open and middle_open and not ring_open and not pinky_open:
        return "Scissors"
    elif index_open and middle_open and ring_open and pinky_open:
        return "Paper"
    else:
        return "Unknown"
```
- This section classifies the gesture based on the state of the fingers:
  - **Rock**: All fingers are closed.
  - **Scissors**: Index and middle fingers are open, but ring and pinky fingers are closed.
  - **Paper**: All fingers are open.
  - If the finger configuration doesn't match any of these patterns, it returns "Unknown."

### 4. **Webcam Capture and Hand Detection:**
```python
cap = cv2.VideoCapture(0)
```
- Opens the webcam feed (`0` indicates the default camera).

### 5. **MediaPipe Hand Detection:**
```python
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
```
- This initializes the **MediaPipe Hands** model with a detection confidence of 0.7 and a tracking confidence of 0.5.
  - **min_detection_confidence**: Minimum confidence value to detect hands.
  - **min_tracking_confidence**: Minimum confidence value to track hand landmarks over time.

### 6. **Main Loop to Process Video Frames:**
```python
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
```
- A loop is started to continuously read frames from the webcam.

```python
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)
```
- The frame is flipped horizontally for a mirror effect.
- The frame is converted from BGR (OpenCV's default) to RGB because MediaPipe requires RGB images.
- `hands.process()` detects hands in the frame and returns the landmarks.

### 7. **Processing Detected Hands:**
```python
if result.multi_hand_landmarks:
    for hand_landmarks in result.multi_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
```
- If hands are detected, the landmarks are drawn on the frame using `mp_drawing.draw_landmarks()`.

```python
        gesture = classify_hand_gesture(hand_landmarks)
        cv2.putText(frame, f'Gesture: {gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
```
- The detected gesture is classified using the `classify_hand_gesture()` function and displayed on the frame.

### 8. **Displaying the Result and Quitting:**
```python
cv2.imshow('Rock, Paper, Scissors Gesture Detection', frame)
if cv2.waitKey(1) & 0xFF == ord('q'):
    break
```
- The frame is displayed with the detected gesture, and the loop continues until the user presses the "q" key to quit.

### 9. **Releasing Resources:**
```python
cap.release()
cv2.destroyAllWindows()
```
- After the loop is exited, the webcam is released, and the OpenCV windows are closed.

### Summary:
- The code captures video from a webcam and uses MediaPipe to detect hand landmarks.
- Based on the finger positions, the code classifies the hand gesture as either "Rock," "Paper," or "Scissors."
- The gesture is displayed on the video feed in real-time.

In [6]:
import cv2
import mediapipe as mp

# Initialize MediaPipe hands and drawing utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to classify gestures
def classify_hand_gesture(hand_landmarks):
    # We will use the tip of the fingers (landmarks: 8, 12, 16, 20) and the thumb (landmark: 4)
    # For simplicity, let's define:
    # Rock: All fingers are closed (folded)
    # Paper: All fingers are open
    # Scissors: Index and middle finger are open, others are closed

    # Get landmarks
    thumb_tip = hand_landmarks.landmark[4].y
    index_tip = hand_landmarks.landmark[8].y
    middle_tip = hand_landmarks.landmark[12].y
    ring_tip = hand_landmarks.landmark[16].y
    pinky_tip = hand_landmarks.landmark[20].y

    # Get the base of each finger (second joint from tip)
    index_base = hand_landmarks.landmark[5].y
    middle_base = hand_landmarks.landmark[9].y
    ring_base = hand_landmarks.landmark[13].y
    pinky_base = hand_landmarks.landmark[17].y

    # Determine if fingers are open (tip is above the base) or closed
    thumb_open = hand_landmarks.landmark[4].x < hand_landmarks.landmark[6].x
    index_open = index_tip < index_base
    middle_open = middle_tip < middle_base
    ring_open = ring_tip < ring_base
    pinky_open = pinky_tip < pinky_base

    # Classify gesture
    if not thumb_open and not index_open and not middle_open and not ring_open and not pinky_open:
        return "Rock"
    elif index_open and middle_open and not ring_open and not pinky_open:
        return "Scissors"
    elif index_open and middle_open and ring_open and pinky_open:
        return "Paper"
    else:
        return "Unknown"

# Initialize webcam
cap = cv2.VideoCapture(1)

# Create MediaPipe hands detector
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a natural selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        result = hands.process(rgb_frame)

        # If hand landmarks are detected
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Classify the gesture
                gesture = classify_hand_gesture(hand_landmarks)

                # Display the classified gesture on the frame
                cv2.putText(frame, f'Gesture: {gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Rock, Paper, Scissors Gesture Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
